This notebook is a supplement to the paper "Negative events - Logical and philosophical aspects"[link] by Lucas Champollion[link] and Timothée Bernard[link].

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s interface to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations.

## Outline

In [60]:
from nltk.test.inference_fixt import setup_module

setup_module()

from nltk import *
#from nltk.sem.drt import DrtParser

from nltk.sem.logic import *
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## Axioms

### Lattice axioms

In [62]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

### Bounded lattice axioms

In [63]:
bot = read_expr('Or(x, Bot) = x')
top = read_expr('And(x, Top) = x')

bounded_lattice = lattice + [bot, top]

### Distributive lattice axioms

In [64]:
dist = read_expr('And(x,Or(y,z)) = Or(And(x,y), And(x,z))')

dist_bounded_lattice = bounded_lattice + [dist]

### Complete lattice axioms

In [65]:
uniq_union = read_expr('all P.(exists z.(all x.(P(x) -> And(x, z) = x) & all y.((all x.(P(x) -> And(x, y) = x)) -> And(y,z) = z)))')
uniq_join = read_expr('all P.(exists z.(all x.(P(x) -> And(x, z) = z) & all y.((all x.(P(x) -> And(x, y) = y)) -> And(y,z) = y)))')

### Boolean algebra axioms

In [66]:
compl = read_expr('all x.(exists y. (And(x, y) = Bot & Or(x, y) = Top))')

#sectional complement
sec_compl = read_expr('all x. (all y. (exists z. (Or(x, z) = y & And(x, z) = Bot)))')

boolean_algebra = dist_bounded_lattice + [uniq_union, uniq_join, compl]

### Exclusion axioms

In [67]:
excl_cumm = read_expr('Excl(x1, y1) & Excl(x2, y2) -> Excl(And(x1, x2), And(y1, y2))')
excl_symm = read_expr('Excl(x, y) <-> Excl(y, x)')

### Possibility axioms

In [68]:
#define conflict:
confl_def = read_expr('confl(x, y) <-> exists z1.(And(z1, x) = x & exists z2.(And(z2, y) = y & Excl(z1, z2)))')

#define possibilities:
int_poss_def = read_expr('int_poss(x) <-> -Excl(x, x)')
ext_poss_def = read_expr('ext_poss(x) <-> exists y.(int_poss(y) & -exists z.(And(y, z) = z & int_poss(z) & -(confl(z,x))))')

poss_def = read_expr('poss(x) <-> -confl(x, x)')

#define world:
#world_def = read_expr('world(x) <-> (int_poss(x) & all y.((And(x, y) = y) -> -int_poss(y)))')

world_def = read_expr('world(x) <-> (poss(x) & all y.((And(x, y) = y) -> -poss(y)))')

#define actual:
actual_def = read_expr('actual(x) <-> world(actual_world) & And(x, actual_world) = x')

#Harmony
#poss_harm = read_expr('all x.(-int_poss(x) -> -ext_poss(x))')

poss_harm = read_expr('all x.(all y. ((world(x) & -confl(x, y)) -> poss(x)))')

#Nirvana
poss_nirv = read_expr('all x.(-confl(Bot, x))')

#Rashōmon
#poss_rshm = read_expr('all x y.((int_poss(x) & int_poss(y) & -confl(x, y)) -> int_poss(And(x,y)))')
poss_rshm = read_expr('all x y.((poss(x) & poss(y) & -confl(x, y)) -> poss(And(x,y)))')


#Cosmopolitanism
#poss_cosm = read_expr('all x.(int_poss(x) -> exists y.(world(y) & And(x, y) = y))')

poss_cosm = read_expr('all x.(poss(x) -> exists y.(world(y) & And(x, y) = x))')

poss_ax = [poss_harm, poss_nirv, poss_rshm, poss_cosm]

#Construct event frame
#choice point: definition of possibility
#choice point: formulation of harmony

event_frame = boolean_algebra + [excl_cumm, excl_symm, confl_def, poss_def, world_def, actual_def]

canonical_frame = event_frame + poss_ax

## Theorems

### Plenitude

In [69]:
#check plenitude
#goal = read_expr('all x.(int_poss(x) -> ext_poss(x))')

goal = read_expr('all x.(poss(x) <-> (exists y.(-confl(x, y) & world(y))))')
prover = Prover9Command(goal, assumptions = canonical_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3310 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:47:35 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 15.
% Level of proof is 5.
% Maximum clause weight is 10.000.
% Given clauses 43.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
11 (all x -confl(Bot,x)).  [assumption].
13 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
24 -world(x) | And(x,y) != y | -poss(y).  [clausify(8)].
27 -poss(x) | world(f9(x)).  [clausify(13)].
33 And(x,y) = And(y,x

### Distributivity of actuality

In [70]:
#check distributivity of actuality
goal = read_expr('all x.((exists y. actual(y) & And(x, y) = x) -> actual(x))')
prover = Prover9Command(goal, assumptions = canonical_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3314 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:47:41 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 11.
% Level of proof is 4.
% Maximum clause weight is 9.000.
% Given clauses 47.


8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
9 actual(x) <-> world(actual_world) & And(x,actual_world) = x.  [assumption].
14 (all x ((exists y actual(y)) & And(x,y) = x -> actual(x))).  [goal].
20 And(x,x) = x.  [assumption].
49 -world(x) | poss(x).  [clausify(8)].
50 -world(x) | And(x,y) != y | -poss(y).  [clausify(8)].
53 -a

In [71]:
#Find counterexample for distributivity of actuality - fail

goal = read_expr('all x.((exists y. actual(y) & And(x, y) = x) -> actual(x))')
mb = MaceCommand(goal, assumptions = canonical_frame)
mb.build_model()

False

### Cumulativity of actuality

In [72]:
#check cumulativity of actuality
goal = read_expr('all P.((all x.(P(x) -> actual(x))) -> (exists z.(all x.(P(x) -> And(x, z) = x) & all y.((all x.(P(x) -> And(x, y) = x)) -> And(y,z) = z)) & actual(z)))')
prover = Prover9Command(goal, assumptions = canonical_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3321 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:47:50 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 14.
% Level of proof is 4.
% Maximum clause weight is 9.000.
% Given clauses 49.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
11 (all x -confl(Bot,x)).  [assumption].
13 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
18 And(x,y) = And(y,x).  [assumption].
48 poss(x) | confl(x,x).  [clausify(7)].
50 -world(x) | And(x,y) != y | -poss(y).  [c

In [52]:
#find counterexample for cumulativity of actuality - fail

goal = read_expr('all P.((all x.(P(x) -> actual(x))) -> (exists z.(all x.(P(x) -> And(x, z) = x) & all y.((all x.(P(x) -> And(x, y) = x)) -> And(y,z) = z)) & actual(z)))')
mb = MaceCommand(goal, assumptions = canonical_frame)
mb.build_model()
print(mb.model('cooked'))

### Manicaeism

In [73]:
#check Manichaeism
goal = read_expr('all x.((world(x) -> all y.(And(x, y) = x | confl(x, y))))')
prover = Prover9Command(goal, assumptions = canonical_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3323 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:47:54 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 13.
% Level of proof is 4.
% Maximum clause weight is 10.000.
% Given clauses 35.


8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
13 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
14 (all x (world(x) -> (all y (And(x,y) = x | confl(x,y))))).  [goal].
23 -world(x) | poss(x).  [clausify(8)].
24 -world(x) | And(x,y) != y | -poss(y).  [clausify(8)].
27 -poss(x) | world(f9(x)).  [clausify

### Classicality of conjunction

In [74]:
#check classicality of conjunction
goal = read_expr(r'((exists x. (P(x) & actual(x))) & (exists y. (Q(y) & actual(y)))) <-> exists y exists z.(actual(y) & actual(z) & P(y) & Q(z) & actual(Or(y, z)))')
prover = Prover9Command(goal, assumptions = canonical_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3327 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:47:57 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 14.
% Level of proof is 4.
% Maximum clause weight is 9.000.
% Given clauses 66.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
11 (all x -confl(Bot,x)).  [assumption].
13 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
18 And(x,y) = And(y,x).  [assumption].
48 poss(x) | confl(x,x).  [clausify(7)].
50 -world(x) | And(x,y) != y | -poss(y).  [c

In [55]:
#check classicality of conjunction
goal = read_expr(r'((exists x. (P(x) & actual(x))) & (exists y. (Q(y) & actual(y)))) <-> exists y exists z.(actual(y) & actual(z) & P(y) & Q(z) & actual(Or(y, z)))')
mb = MaceCommand(goal, assumptions = canonical_frame)
mb.build_model()
print(mb.model('cooked'))

### Classicality of disjunction

In [75]:
#check classicality of disjunction
goal = read_expr(r'(exists x. (actual(x) & (P(x) | Q(x)))) <-> ((exists x. (actual(x) & P(x))) & (exists y. (actual(y) & Q(y))))')
prover = Prover9Command(goal, assumptions = canonical_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3329 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:48:01 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 14.
% Level of proof is 4.
% Maximum clause weight is 9.000.
% Given clauses 54.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
11 (all x -confl(Bot,x)).  [assumption].
13 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
18 And(x,y) = And(y,x).  [assumption].
48 poss(x) | confl(x,x).  [clausify(7)].
50 -world(x) | And(x,y) != y | -poss(y).  [c

### No Gaps

In [77]:
#define negating a set
def negate(x, P):
    return '(all a. (P(a) -> (exists b. And(x,b) = b & Excl(a, b))) & (all c. ((And(c, x) = c & (-exists d. (And(x, d) = d & And(c, d) = d))) -> (exists d. (P(d) & Excl(d, c))))) & all e all f. ((e != f) -> (exists g exists h.(And(g, x) = g & And(h, x) = h & g!=h & Excl(e, g) & Excl(f, h)))))'

#check no gaps
goal = read_expr('(-exists x. (actual(x) & P(x))) <-> (exists y. (actual(y) &'+negate('y', 'P')+'))')
prover = Prover9Command(goal, assumptions = canonical_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3335 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:48:15 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 14.
% Level of proof is 4.
% Maximum clause weight is 9.000.
% Given clauses 114.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
11 (all x -confl(Bot,x)).  [assumption].
13 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
18 And(x,y) = And(y,x).  [assumption].
48 poss(x) | confl(x,x).  [clausify(7)].
50 -world(x) | And(x,y) != y | -poss(y).  [

### No Gluts

In [78]:
#check no gluts
goal = read_expr('-((exists x. actual(x) & P(x)) & (exists y. actual(y) &'+negate('y', 'P')+'))')
prover = Prover9Command(goal, assumptions = canonical_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3339 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:48:17 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 16.
% Level of proof is 5.
% Maximum clause weight is 10.000.
% Given clauses 58.


8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
9 actual(x) <-> world(actual_world) & And(x,actual_world) = x.  [assumption].
13 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
14 -((exists x actual(x)) & P(x) & (exists y actual(y)) & (all a (P(a) -> (exists b And(x,b) = b) & Excl(a,b))) & (all c (And(c,

## Independence of axioms

In [84]:
#Harmony is derivable from other possibility axioms

goal = poss_harm
prover = Prover9Command(goal, assumptions = [ax for ax in canonical_frame if not (ax == poss_harm | ax == poss_rshm)], timeout = 1000)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3382 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:49:59 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 3.
% Maximum clause weight is 2.000.
% Given clauses 0.


8 world(x) <-> poss(x) & (all y (And(x,y) = y -> -poss(y))).  [assumption].
14 (all x all y (world(x) & -confl(x,y) -> poss(x))).  [goal].
23 -world(x) | poss(x).  [clausify(8)].
28 world(c1).  [deny(14)].
63 -poss(c1).  [deny(14)].
72 poss(c1).  [resolve(28,a,23,a)].
73 $F.  [copy(72),unit_del(a,63)].

============================== end of proof =============

In [80]:
#Nirvana is not derivable from other possibility axioms (counterexample found)
goal = poss_nirv
mb = MaceCommand(goal, assumptions = [ax for ax in canonical_frame if not (ax == poss_nirv)])
mb.build_model()
print(mb.model(format = 'cooked'))

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

actual_world = 0.

c1 = 0.

f1(0) = 0.
f1(1) = 0.

f3(0) = 0.
f3(1) = 0.

f5(0) = 1.
f5(1) = 0.

f8(0) = 0.
f8(1) = 0.

f9(0) = 0.
f9(1) = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

f2(0,0) = 0.
f2(0,1) = 0.
f2(1,0) = 0.
f2(1,1) = 0.

f4(0,0) = 0.
f4(0,1) = 0.
f4(1,0) = 0.
f4(1,1) = 0.

f6(0,0) = 0.
f6(0,1) = 0.
f6(1,0) = 1.
f6(1,1) = 1.

f7(0,0) = 0.
f7(0,1) = 1.
f7(1,0) = 0.
f7(1,1) = 1.

  P(0).
- P(1).

- actual(0).
- actual(1).

- poss(0).
- poss(1).

- world(0).
- world(1).

  Excl(0,0).
  Excl(0,1).
  Excl(1,0).
  Excl(1,1).

  confl(0,0).
  confl(0,1).
  confl(1,0).
  confl(1,1).



In [81]:
#Rashōmon is derivable from other possibility axioms
goal = poss_rshm
prover = Prover9Command(goal, assumptions = [ax for ax in canonical_frame if not (ax == poss_rshm | ax == poss_harm)], timeout = 1000)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 3349 was started by guzzler on oahzzdepbm,
Mon Jul  4 19:48:34 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 2.
% Maximum clause weight is 11.000.
% Given clauses 30.


12 (all x all y (poss(x) & poss(y) & -confl(x,y) -> poss(And(x,y)))).  [assumption].
14 (all x all y (poss(x) & poss(y) & -confl(x,y) -> poss(And(x,y)))).  [goal].
59 -poss(x) | -poss(y) | confl(x,y) | poss(And(x,y)).  [clausify(12)].
61 poss(c1).  [deny(14)].
62 poss(c2).  [deny(14)].
63 -confl(c1,c2).  [deny(14)].
64 -poss(And(c1,c2)).  [deny(14)].
125 $F.

In [83]:
#Cosmopolitan is not derivable from other possibility axioms (counterexample found)
goal = poss_cosm
mb = MaceCommand(goal, assumptions = [ax for ax in canonical_frame if not (ax == poss_cosm)])
mb.build_model()
print(mb.model(format = 'cooked'))

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

actual_world = 0.

c1 = 0.

f1(0) = 0.
f1(1) = 0.

f3(0) = 0.
f3(1) = 0.

f5(0) = 1.
f5(1) = 0.

f8(0) = 0.
f8(1) = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

f2(0,0) = 0.
f2(0,1) = 0.
f2(1,0) = 0.
f2(1,1) = 0.

f4(0,0) = 0.
f4(0,1) = 0.
f4(1,0) = 0.
f4(1,1) = 0.

f6(0,0) = 0.
f6(0,1) = 0.
f6(1,0) = 0.
f6(1,1) = 0.

f7(0,0) = 0.
f7(0,1) = 0.
f7(1,0) = 0.
f7(1,1) = 0.

  P(0).
- P(1).

- actual(0).
- actual(1).

  poss(0).
  poss(1).

- world(0).
- world(1).

- Excl(0,0).
- Excl(0,1).
- Excl(1,0).
- Excl(1,1).

- confl(0,0).
- confl(0,1).
- confl(1,0).
- confl(1,1).

